# PREVISÃO DE DEMANDA UTILIZANDO CATBOOST E HYPERPARAMETER TUNNING




Importando as bibliotecas necessárias

In [ ]:
import pandas as pd
import numpy as np

from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import plotly_express as px
%matplotlib inline
import plotly.offline as py
py.init_notebook_mode(connected=True)
import xgboost as xgb
import lightgbm as lightgbm
from catboost import Pool, CatBoostRegressor
import shap
import warnings
# import the_module_that_warns

warnings.filterwarnings("ignore")

from fbprophet import Prophet
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
import missingno as msno

Carregando os dados de Treino

In [ ]:
df_raw_train = pd.read_csv("../input/predict-demand/train.csv")
df_raw_train.head()

In [ ]:
df_raw_train.info()

In [ ]:
msno.matrix(df_raw_train)

A base de dados possui alguns nulos nos atributos "lat" e "long"

Avaliando a distribuição das vendas nota-se que os dados estão "enviesados" à direita

In [ ]:
fig = px.histogram(df_raw_train['quantity'],title='Distribuição dos dados de Venda Agrupados')
fig.show()

In [ ]:
df_raw_train[['price','quantity']].describe()

In [ ]:
df_raw_train.columns

In [ ]:
df1 = df_raw_train.copy()

In [ ]:
df1.isna().sum()

Ausência de registros em vários dados (exclusão dos dados de teste) 

In [ ]:
df1 = df1.dropna()

In [ ]:
df1.isna().sum()

Calculando o ranking de vendas por Shop

In [ ]:
df1.groupby('shop')['quantity'].sum().sort_values(ascending=False)

Analisando a distribuição dos dados de venda por Shop através de Box Plot

In [ ]:
fig = px.box(df1,y='quantity',x='shop',title='Dispersão de Venda por Shop')
fig.show()

Analisando o ranking de vendas por Brand

In [ ]:
df1.groupby('brand')['quantity'].sum().sort_values(ascending=False)

Avaliando a dispersão dos dados de vendas por Brand utilizando Box Plot

In [ ]:
fig = px.box(df1,y='quantity',x='brand',title='Dispersão de Venda por Brand')
fig.show()

Analisando ranking de vendas por City

In [ ]:
df1.groupby('city')['quantity'].sum().sort_values(ascending=False)

Avaliando a dispersão de vendas por City através do gráfico Box Plot

In [ ]:
fig = px.box(df1,y='quantity',x='city',title='Dispersão de Venda por Brand')
fig.show()

Criação de funções e variáveis de apoio

In [ ]:
def rmse(ytrue, ypred):
    return np.sqrt(mean_squared_error(ytrue, ypred))

In [ ]:
df1['date']=pd.to_datetime(df1['date'])
print('Data inicio: {}   Data fim: {}'.format(df1['date'].min(),df1['date'].max()))

Analisando a evolução de vendas por mês verifica-se a presença de sazonalidade anual com pico de venda em Julho e vale em Fevereiro

In [ ]:
dfgroup = df1.groupby('date')['quantity'].sum()
px.line(dfgroup)

Criação de variáveis de calendário

In [ ]:
df1['day']=df1['date'].dt.day
df1['week']=df1['date'].dt.week
df1['month']=df1['date'].dt.month
df1['year']=df1['date'].dt.year
df1['dow']=df1['date'].dt.dayofweek

In [ ]:
df1

# SPLIT DE TREINO E VALIDAÇÃO

In [ ]:
df1.columns

In [ ]:
columns=['id','lat','long','pop']
df2 = df1.drop(columns=columns)
df2.head()

In [ ]:
traindf2 = df2[(df2['date']<'2017-07-01')]
valdf2 = df2[(df2['date']>='2017-07-01')]

traindf2 = traindf2.drop(columns=['date'])
valdf2 = valdf2.drop(columns=['date'])

In [ ]:
xtr, xval = traindf2.drop(['quantity'], axis=1), valdf2.drop(['quantity'], axis=1)
ytr, yval = traindf2['quantity'].values, valdf2['quantity'].values

In [ ]:
xval

# MODELAGEM DE DADOS UTILIZANDO CATBOOST REGRESSOR

In [ ]:
cfi = ['shop','city','brand','container','capacity']

In [ ]:
model = CatBoostRegressor(
    n_estimators = 2000,
    learning_rate= 0.1,
    max_depth=6,
    loss_function = 'RMSE',
    eval_metric = 'RMSE',
    l2_leaf_reg=10,
    cat_features = cfi)

In [ ]:
model.fit( xtr, ytr, eval_set=(xval,yval), silent=True, plot=True )

In [ ]:
model.plot_tree(
    tree_idx=0,
    pool=None
)

Registro da aderência dos modelos de acordo com hyperparametros

* CatBoostRegressor(
    n_estimators = 1000,
    learning_rate= 0.1,
    max_depth=6,
    loss_function = 'RMSE',
    eval_metric = 'RMSE',
    l2_leaf_reg=9,
    cat_features = cfi)
    BEST TEST: 3169.792976

#################################################

* model = CatBoostRegressor(
    n_estimators = 2000,
    learning_rate= 0.1,
    max_depth=6,
    loss_function = 'RMSE',
    eval_metric = 'RMSE',
    l2_leaf_reg=10,
    cat_features = cfi)
    BEST MODEL: 2949.235413


In [ ]:
p = model.predict(xval)
print('Raiz Quadrada do Erro Médio Quadrático na base de TESTE: {}'.format(rmse(p,yval)))

Calculando a correlação de Pearson entre Previsão e Venda - análise de aderência da previsão gerada 

In [ ]:
np.corrcoef(p,yval)[1,0]

# GRIDSEARCH PARA HYPERPARAMETER TUNING

model = CatBoostRegressor(cat_features = cfi)

grid = {'n_estimators':[2000],
        'learning_rate': [0.1, 0.5, 1],
        'l2_leaf_reg': [10],
       'max_depth':[6,10,15]}

grid_search_result = model.grid_search(grid, 
                                       X=xtr, 
                                       y=ytr,
                                       verbose= False,
                                       plot=True)

# INTERPRETANDO OS RESULTADOS - IMPORTÂNCIA DAS VARIÁVEIS UTILIZANDO SHAP

In [ ]:
shap.initjs()
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(Pool(xtr, ytr, cat_features=cfi))
shap.force_plot(explainer.expected_value, shap_values[0,:], xtr.iloc[0,:])

In [ ]:
# visualize the training set predictions
shap.force_plot(explainer.expected_value, shap_values[0:50,:], xtr.iloc[0:50,:])

In [ ]:
# summarize the effects of all the features
shap.summary_plot(shap_values, xtr)

In [ ]:
# feature importance plot
shap.summary_plot(shap_values, xtr, plot_type="bar",title='Feature Importance')


# PRÓXIMOS PASSOS

* Utilizar outros modelos e comparar resultados (Linear Regression, Prophet, Sarimax, LSTM, etc)
* Segmentação dos dados através de clusterização para maior aderência dos modelos

# REFERÊNCIAS

https://towardsdatascience.com/deep-dive-into-catboost-functionalities-for-model-interpretation-7cdef669aeed

https://www.kaggle.com/mcsweena/feature-importance-with-catboost-and-shap**